<a href="https://colab.research.google.com/github/QSLV/personalized_fashion_recs/blob/main/feature_construct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install kaggle
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
!wget https://raw.githubusercontent.com/benhamner/Metrics/master/Python/ml_metrics/average_precision.py

--2022-04-25 01:51:47--  https://raw.githubusercontent.com/benhamner/Metrics/master/Python/ml_metrics/average_precision.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1654 (1.6K) [text/plain]
Saving to: ‘average_precision.py’

average_precision.p 100%[===================>]   1.62K  --.-KB/s    in 0s      

2022-04-25 01:51:47 (23.6 MB/s) - ‘average_precision.py’ saved [1654/1654]



In [2]:
# Load Packages
import pandas as pd
import numpy as np
from lightgbm.sklearn import LGBMRanker
from average_precision import apk, mapk
import random
from sklearn.base import BaseEstimator, TransformerMixin

## 读取数据

In [5]:
%%time
transactions = pd.read_parquet('/content/drive/MyDrive/H&M/data/transactions_train.parquet')
customers = pd.read_parquet('/content/drive/MyDrive/H&M/data/customers.parquet')
articles = pd.read_parquet('/content/drive/MyDrive/H&M/data/articles.parquet')

CPU times: user 2.1 s, sys: 1.81 s, total: 3.91 s
Wall time: 2.99 s


In [6]:
display(transactions.head())
display(customers.head())
display(articles.head())


,t_dat,customer_id,article_id,price,sales_channel_id,week
25784,2018-09-20,1728846800780188,519773001,0.028458,2,0
25785,2018-09-20,1728846800780188,578472001,0.032525,2,0
5389,2018-09-20,2076973761519164,661795002,0.167797,2,0
5390,2018-09-20,2076973761519164,684080003,0.101678,2,0
47429,2018-09-20,2918879973994241,662980001,0.033881,1,0


,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,6883939031699146327,-1,-1,0,0,49,6305
1,11246327431398957306,-1,-1,0,0,25,33726
2,18439897732908966680,-1,-1,0,0,24,3247
3,18352672461570950206,-1,-1,0,0,54,168643
4,18162778555210377306,1,1,0,1,52,168645


,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,12855,253,9,0,1010016,0,9,0,...,10,0,0,1,0,16,30,1002,2,8834
1,108775044,108775,12855,253,9,0,1010016,0,10,2,...,10,0,0,1,0,16,30,1002,2,8834
2,108775051,108775,44846,253,9,0,1010017,3,11,11,...,10,0,0,1,0,16,30,1002,2,8834
3,110065001,110065,8159,306,13,4,1010016,0,9,0,...,131,7,7,1,0,61,5,1017,4,8243
4,110065002,110065,8159,306,13,4,1010016,0,10,2,...,131,7,7,1,0,61,5,1017,4,8243


In [7]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype
---  ------                        --------------   -----
 0   article_id                    105542 non-null  int32
 1   product_code                  105542 non-null  int32
 2   prod_name                     105542 non-null  int32
 3   product_type_no               105542 non-null  int32
 4   product_type_name             105542 non-null  int16
 5   product_group_name            105542 non-null  int8 
 6   graphical_appearance_no       105542 non-null  int32
 7   graphical_appearance_name     105542 non-null  int8 
 8   colour_group_code             105542 non-null  int32
 9   colour_group_name             105542 non-null  int8 
 10  perceived_colour_value_id     105542 non-null  int32
 11  perceived_colour_value_name   105542 non-null  int8 
 12  perceived_colour_master_id    105542 non-null  int32
 13  perceived_colo

In [6]:
len(transactions.article_id.unique()), len(articles)

(104547, 31788324)

In [9]:
all_df = pd.merge(transactions, articles, on='article_id', how='left')
all_df = pd.merge(all_df, customers, on='customer_id', how='left')


In [10]:
all_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id,week,product_code,prod_name,product_type_no,product_type_name,...,section_name,garment_group_no,garment_group_name,detail_desc,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,2018-09-20,1728846800780188,519773001,0.028458,2,0,519773,7147,245,17,...,0,1003,3,10231,-1,-1,0,0,59,44730
1,2018-09-20,1728846800780188,578472001,0.032525,2,0,578472,37340,263,38,...,40,1007,9,26053,-1,-1,0,0,59,44730
2,2018-09-20,2076973761519164,661795002,0.167797,2,0,661795,43993,263,38,...,40,1007,9,32892,-1,-1,0,0,55,18589
3,2018-09-20,2076973761519164,684080003,0.101678,2,0,684080,1768,262,6,...,40,1007,9,6151,-1,-1,0,0,55,18589
4,2018-09-20,2918879973994241,662980001,0.033881,1,0,662980,2110,252,2,...,0,1003,3,8172,-1,-1,0,0,40,22815


# 特征构建

In [11]:
def group_func(df, group_func_dic, group_key):
  if isinstance(group_func_dic, str):
    group_func_dic = [group_func_dic]
  features = df.groupby(group_key).agg(group_func_dic)
  features.columns = [e[0] + "_" + e[1].upper() for e in features.columns.tolist()]
  features.reset_index(inplace=True)
  return features
  
def add_prefix(df, exclude_columns, prefix):
    if isinstance(exclude_columns, str):
        exclude_columns = [exclude_columns]
        
    column_names = [col for col in df.columns if col not in exclude_columns]
    df.rename(columns = dict(zip(column_names, [prefix + name for name in column_names])), inplace=True)
    return df

## 用户侧

In [12]:
# 用户基础统计特征
def get_user_feature(df):
  feature_type = {
      'customer_id': ['count'],
      'article_id': ['nunique'],
      'price': ['max', 'min', 'mean']
  }
  feature = group_func(df, feature_type, group_key=['customer_id'])
  feature = add_prefix(feature, ['customer_id'], 'customer_')
  return feature

In [13]:
user_feature = get_user_feature(all_df)
user_feature.head(10)

,customer_id,customer_customer_id_COUNT,customer_article_id_NUNIQUE,customer_price_MAX,customer_price_MIN,customer_price_MEAN
0,4245900472157,2,2,0.025407,0.016932,0.021169
1,23962613628581,20,18,0.067780,0.006763,0.036932
2,25398598941468,1,1,0.042356,0.042356,0.042356
3,28847241659200,74,71,0.128797,0.004051,0.033151
4,41046458195168,4,2,0.016932,0.016932,0.016932
5,41318098387474,32,31,0.033881,0.008458,0.019369
6,43757848922470,16,16,0.118627,0.005068,0.029212
7,46878247658203,1,1,0.016932,0.016932,0.016932
8,56938741737809,12,12,0.036000,0.003373,0.014284
9,77117344919861,294,216,0.118627,0.005746,0.034021
